In [1]:
!pip install transformers
!pip install huggingface_hub
!pip install datasets
!pip install evaluate
# !pip install torchtext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 35.0 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.6 MB/s eta 0:00:00


In [2]:
!pip install spacy
!python -m spacy download fr_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 81.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [3]:
# Personal Transformer
import spacy
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.nn.utils.rnn import pad_sequence
import torch
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
raw_datasets = load_dataset("ashwinradhe/dl_dataset")
small_train_dataset = raw_datasets['train'].select(range(100000))
SRC_LANGUAGE = "en"
TRG_LANGUAGE = "fr"
tokenizer_en = get_tokenizer('spacy', language='en_core_web_sm')
tokenizer_fr= get_tokenizer('spacy', language='fr_core_news_sm')
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']
def tokenize_and_build_vocab(data, tokenizer):
    tokenized_data = [tokenizer(sentence) for sentence in data]
    vocab = build_vocab_from_iterator(tokenized_data,min_freq=1, specials=special_symbols,special_first=True)
    vocab.set_default_index(vocab['<unk>'])
    return vocab, tokenized_data
vocab_en, tokenized_data_en = tokenize_and_build_vocab(small_train_dataset['English Sentence'], tokenizer_en)
vocab_fr, tokenized_data_fr = tokenize_and_build_vocab(small_train_dataset['French Sentence'], tokenizer_fr)
def numericalize_and_pad(tokenized_data, vocab):
    numericalized_data = [torch.tensor(vocab.lookup_indices(sentence)) for sentence in tokenized_data]
    padded_data = pad_sequence(numericalized_data, batch_first=True, padding_value=vocab['<pad>'])
    return padded_data
padded_data_en = numericalize_and_pad(tokenized_data_en, vocab_en)
padded_data_fr = numericalize_and_pad(tokenized_data_fr, vocab_fr)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/1825077 [00:00<?, ? examples/s]

In [4]:
from torch.utils.data import random_split
from torch.utils.data import DataLoader
dataset = torch.utils.data.TensorDataset(padded_data_en, padded_data_fr)
total_size = len(dataset)
train_size = int(0.8 * total_size)
val_size = int(0.1 * total_size)
test_size = total_size - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

batch_size = 32

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, drop_last=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, drop_last=True)

In [15]:
import torch
from torch import nn, Tensor
import math

class PositionalEncoding(nn.Module):
    def __init__(self, emb_size: int, dropout: float, maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(-torch.arange(0, emb_size, 2) * math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size: int):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

class TransformerModel(nn.Module):
    def __init__(self, num_tokens_en, num_tokens_fr, embed_size, nhead, dim_feedforward, max_seq_length):
        super(TransformerModel, self).__init__()
        self.embed_size = embed_size
        self.src_tok_emb = TokenEmbedding(num_tokens_en, embed_size)
        self.tgt_tok_emb = TokenEmbedding(num_tokens_fr, embed_size)
        self.positional_encoding = PositionalEncoding(embed_size, dropout=0.1, maxlen=max_seq_length)

        encoder_layer = nn.TransformerEncoderLayer(d_model=embed_size, nhead=nhead, dim_feedforward=dim_feedforward, dropout=0.1)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=3)

        decoder_layer = nn.TransformerDecoderLayer(d_model=embed_size, nhead=nhead, dim_feedforward=dim_feedforward, dropout=0.1)
        self.transformer_decoder = nn.TransformerDecoder(decoder_layer, num_layers=3)
        self.generator = nn.Linear(embed_size, num_tokens_fr)

    def encode(self, src, src_mask):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        return self.transformer_encoder(src_emb, src_key_padding_mask=src_mask)

    def decode(self, tgt, memory, tgt_mask, tgt_key_padding_mask):
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(tgt))
        return self.transformer_decoder(tgt_emb, memory, tgt_mask, tgt_key_padding_mask=tgt_key_padding_mask)

    def forward(self, src, tgt, src_mask=None, tgt_mask=None, src_padding_mask=None, tgt_padding_mask=None):
        memory = self.encode(src, src_padding_mask)
        output = self.decode(tgt, memory, tgt_mask, tgt_padding_mask)
        return self.generator(output)

    def generate_square_subsequent_mask(self, sz):
        mask = torch.triu(torch.ones(sz, sz, device=device)).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask
model_sgd_n = TransformerModel(len(vocab_en), len(vocab_fr), embed_size=64, nhead=8, dim_feedforward=1024, max_seq_length=512)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [ ]:
model.load_state_dict(torch.load('/content/drive/MyDrive/Adam/absolute_positional_encoding/crossentropyloss/model_weights.pt'))

<All keys matched successfully>

In [ ]:
from torchtext.data.metrics import bleu_score
import evaluate
seq2seq_loss = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)
def remove_repetitions(sentence):
    tokens = sentence.split()
    previous_token = None
    filtered_tokens = []
    for token in tokens:
        if token != previous_token:
            filtered_tokens.append(token)
            previous_token = token
    return ' '.join(filtered_tokens)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
def test_and_evaluate(model, test_dataloader, loss_fn, vocab_en, vocab_fr, num_examples=10):
    model.eval()
    total_test_loss = 0
    examples_printed = 0
    meteor = evaluate.load('meteor')
    references = []
    candidates = []
    with torch.no_grad():
      for src, tgt in test_dataloader:
          src, tgt = src.to(device), tgt.to(device)
          src = src.T
          tgt_inp = tgt.T
          tgt_input = tgt_inp[:-1, :]
          tgt_output = tgt_inp[1:, :]
          src_seq_length = src.shape[0]
          tgt_seq_length = tgt_input.shape[0]
          src_mask = torch.zeros(src_seq_length, src_seq_length).to(device).type(torch.bool)
          tgt_mask = model.generate_square_subsequent_mask(tgt_seq_length).to(device)
          src_padding_mask = (src == PAD_IDX).transpose(0, 1).to(device)
          tgt_padding_mask = (tgt_input == PAD_IDX).transpose(0, 1).to(device)

          logits = model(src, tgt_input, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask)
          loss = loss_fn(logits.view(-1, logits.shape[-1]), tgt_output.reshape(-1))
          total_test_loss += loss.item()
          if examples_printed < num_examples:
              predictions = logits.argmax(dim=-1)
              for i in range(predictions.size(1)):
                  if examples_printed >= num_examples:
                      break
                  # input_sentence = ' '.join([vocab_en.get_itos()[idx] for idx in src[:, i] if idx not in (vocab_en['PAD_IDX'], vocab_en['<BOS>'], vocab_en['<EOS>'])])
                  input_sentence = ' '.join([vocab_en.get_itos()[idx] for idx in src[:, i] if idx not in (vocab_en['<pad>'], vocab_en['<bos>'], vocab_en['<eos>'])])
                  target_sentence = ' '.join([vocab_fr.get_itos()[idx] for idx in tgt[:, i] if idx not in (vocab_fr['<pad>'], vocab_fr['<bos>'], vocab_fr['<eos>'])])
                  # predicted_sentence = ' '.join([vocab_fr.get_itos()[idx] for idx in predictions[:, i] if idx not in (vocab_fr['<pad>'], vocab_fr['<bos>'], vocab_fr['<eos>'])])
                  # translated
                  predicted_sentence = remove_repetitions(' '.join([vocab_fr.get_itos()[idx] for idx in predictions[:, i] if idx not in (vocab_fr['<pad>'], vocab_fr['<bos>'], vocab_fr['<eos>'])]))
                  predicted_sentence_with_repititions = ' '.join([vocab_fr.get_itos()[idx] for idx in predictions[:, i] if idx not in (vocab_fr['<pad>'], vocab_fr['<bos>'], vocab_fr['<eos>'])])
                  #print(f'Input: {input_sentence}')
                  #print(f'Target: {target_sentence}')
                  #print(f'Predicted: {predicted_sentence}')
                  #print(f'Predicted with repetitions: {predicted_sentence_with_repititions}')
                  #print()
                  examples_printed += 1
                  references.append([target_sentence])
                  candidates.append(predicted_sentence)
    meteor_score = meteor.compute(predictions=candidates, references=references)
    print(f'Meteor Score: {meteor_score}')
    avg_test_loss = total_test_loss / len(test_dataloader)
    print(f'Test Loss: {avg_test_loss:.4f}')
    return avg_test_loss
test_loss = test_and_evaluate(model, test_dataloader, seq2seq_loss, vocab_en, vocab_fr)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Meteor Score: {'meteor': 0.08188700818458425}
Test Loss: 3.3265


In [ ]:
model_cel.load_state_dict(torch.load('/content/drive/MyDrive/Adam/absolute_positional_encoding/crossentropywithlabelsmoothning/model_cel_state_dict.pkl'))

<All keys matched successfully>

In [ ]:
from torchtext.data.metrics import bleu_score
import evaluate
seq2seq_loss_ls = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX,label_smoothing=0.1)
def remove_repetitions(sentence):
    tokens = sentence.split()
    previous_token = None
    filtered_tokens = []
    for token in tokens:
        if token != previous_token:
            filtered_tokens.append(token)
            previous_token = token
    return ' '.join(filtered_tokens)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_cel.to(device)
def test_and_evaluate(model, test_dataloader, loss_fn, vocab_en, vocab_fr, num_examples=10):
    model.eval()
    total_test_loss = 0
    examples_printed = 0
    meteor = evaluate.load('meteor')
    references = []
    candidates = []
    with torch.no_grad():
      for src, tgt in test_dataloader:
          src, tgt = src.to(device), tgt.to(device)
          src = src.T
          tgt_inp = tgt.T
          tgt_input = tgt_inp[:-1, :]
          tgt_output = tgt_inp[1:, :]
          src_seq_length = src.shape[0]
          tgt_seq_length = tgt_input.shape[0]
          src_mask = torch.zeros(src_seq_length, src_seq_length).to(device).type(torch.bool)
          tgt_mask = model.generate_square_subsequent_mask(tgt_seq_length).to(device)
          src_padding_mask = (src == PAD_IDX).transpose(0, 1).to(device)
          tgt_padding_mask = (tgt_input == PAD_IDX).transpose(0, 1).to(device)

          logits = model(src, tgt_input, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask)
          loss = loss_fn(logits.view(-1, logits.shape[-1]), tgt_output.reshape(-1))
          total_test_loss += loss.item()

          if examples_printed < num_examples:
              predictions = logits.argmax(dim=-1)
              for i in range(predictions.size(1)):
                  if examples_printed >= num_examples:
                      break
                  # input_sentence = ' '.join([vocab_en.get_itos()[idx] for idx in src[:, i] if idx not in (vocab_en['PAD_IDX'], vocab_en['<BOS>'], vocab_en['<EOS>'])])
                  input_sentence = ' '.join([vocab_en.get_itos()[idx] for idx in src[:, i] if idx not in (vocab_en['<pad>'], vocab_en['<bos>'], vocab_en['<eos>'])])
                  target_sentence = ' '.join([vocab_fr.get_itos()[idx] for idx in tgt[:, i] if idx not in (vocab_fr['<pad>'], vocab_fr['<bos>'], vocab_fr['<eos>'])])
                  # predicted_sentence = ' '.join([vocab_fr.get_itos()[idx] for idx in predictions[:, i] if idx not in (vocab_fr['<pad>'], vocab_fr['<bos>'], vocab_fr['<eos>'])])
                  # translated
                  predicted_sentence = remove_repetitions(' '.join([vocab_fr.get_itos()[idx] for idx in predictions[:, i] if idx not in (vocab_fr['<pad>'], vocab_fr['<bos>'], vocab_fr['<eos>'])]))
                  predicted_sentence_with_repititions = ' '.join([vocab_fr.get_itos()[idx] for idx in predictions[:, i] if idx not in (vocab_fr['<pad>'], vocab_fr['<bos>'], vocab_fr['<eos>'])])
                  #print(f'Input: {input_sentence}')
                  #print(f'Target: {target_sentence}')
                  #print(f'Predicted: {predicted_sentence}')
                  #print(f'Predicted with repetitions: {predicted_sentence_with_repititions}')
                  #print()
                  examples_printed += 1
                  references.append([target_sentence])
                  candidates.append(predicted_sentence)
    meteor_score = meteor.compute(predictions=candidates, references=references)
    print(f'Meteor Score: {meteor_score}')
    avg_test_loss = total_test_loss / len(test_dataloader)
    print(f'Test Loss: {avg_test_loss:.4f}')
    return avg_test_loss
test_loss_ls = test_and_evaluate(model_cel, test_dataloader, seq2seq_loss_ls, vocab_en, vocab_fr)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:5109: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


Meteor Score: {'meteor': 0.07580169478000488}
Test Loss: 4.8215


In [6]:
model_sc.load_state_dict(torch.load('/content/drive/MyDrive/Adam/absolute_positional_encoding/sparsecategoricalcrossentropy/model_weights_sc.pkl'))

<All keys matched successfully>

In [7]:
from torchtext.data.metrics import bleu_score
import evaluate
seq2seq_loss_n = nn.NLLLoss(ignore_index=PAD_IDX)
def remove_repetitions(sentence):
    tokens = sentence.split()
    previous_token = None
    filtered_tokens = []
    for token in tokens:
        if token != previous_token:
            filtered_tokens.append(token)
            previous_token = token
    return ' '.join(filtered_tokens)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_sc.to(device)
def test_and_evaluate(model, test_dataloader, loss_fn, vocab_en, vocab_fr, num_examples=10):
    model.eval()
    total_test_loss = 0
    examples_printed = 0
    meteor = evaluate.load('meteor')
    references = []
    candidates = []
    with torch.no_grad():
      for src, tgt in test_dataloader:
          src, tgt = src.to(device), tgt.to(device)
          src = src.T
          tgt_inp = tgt.T
          tgt_input = tgt_inp[:-1, :]
          tgt_output = tgt_inp[1:, :]
          src_seq_length = src.shape[0]
          tgt_seq_length = tgt_input.shape[0]
          src_mask = torch.zeros(src_seq_length, src_seq_length).to(device).type(torch.bool)
          tgt_mask = model.generate_square_subsequent_mask(tgt_seq_length).to(device)
          src_padding_mask = (src == PAD_IDX).transpose(0, 1).to(device)
          tgt_padding_mask = (tgt_input == PAD_IDX).transpose(0, 1).to(device)

          logits = model(src, tgt_input, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask)
          logits = torch.nn.functional.log_softmax(logits, dim=-1)
          loss = loss_fn(logits.view(-1, logits.shape[-1]), tgt_output.reshape(-1))
          total_test_loss += loss.item()
          if examples_printed < num_examples:
              predictions = logits.argmax(dim=-1)
              for i in range(predictions.size(1)):
                  if examples_printed >= num_examples:
                      break
                  # input_sentence = ' '.join([vocab_en.get_itos()[idx] for idx in src[:, i] if idx not in (vocab_en['PAD_IDX'], vocab_en['<BOS>'], vocab_en['<EOS>'])])
                  input_sentence = ' '.join([vocab_en.get_itos()[idx] for idx in src[:, i] if idx not in (vocab_en['<pad>'], vocab_en['<bos>'], vocab_en['<eos>'])])
                  target_sentence = ' '.join([vocab_fr.get_itos()[idx] for idx in tgt[:, i] if idx not in (vocab_fr['<pad>'], vocab_fr['<bos>'], vocab_fr['<eos>'])])
                  # predicted_sentence = ' '.join([vocab_fr.get_itos()[idx] for idx in predictions[:, i] if idx not in (vocab_fr['<pad>'], vocab_fr['<bos>'], vocab_fr['<eos>'])])
                  # translated
                  predicted_sentence = remove_repetitions(' '.join([vocab_fr.get_itos()[idx] for idx in predictions[:, i] if idx not in (vocab_fr['<pad>'], vocab_fr['<bos>'], vocab_fr['<eos>'])]))
                  predicted_sentence_with_repititions = ' '.join([vocab_fr.get_itos()[idx] for idx in predictions[:, i] if idx not in (vocab_fr['<pad>'], vocab_fr['<bos>'], vocab_fr['<eos>'])])
                  #print(f'Input: {input_sentence}')
                  #print(f'Target: {target_sentence}')
                  #print(f'Predicted: {predicted_sentence}')
                  #print(f'Predicted with repetitions: {predicted_sentence_with_repititions}')
                  #print()
                  examples_printed += 1
                  references.append([target_sentence])
                  candidates.append(predicted_sentence)
    meteor_score = meteor.compute(predictions=candidates, references=references)
    print(f'Meteor Score: {meteor_score}')
    avg_test_loss = total_test_loss / len(test_dataloader)
    print(f'Test Loss: {avg_test_loss:.4f}')
    return avg_test_loss
test_loss_ls = test_and_evaluate(model_sc, test_dataloader, seq2seq_loss_n, vocab_en, vocab_fr)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:5109: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


Meteor Score: {'meteor': 0.12180915044027014}
Test Loss: 3.4901


In [9]:
model_adamw.load_state_dict(torch.load('/content/drive/MyDrive/Adamw/absolute_positional_encoding/crossentropyloss/model_metrics_adamw.pkl'))

<All keys matched successfully>

In [10]:
from torchtext.data.metrics import bleu_score
import evaluate
seq2seq_loss_adamw = nn.CrossEntropyLoss(ignore_index=PAD_IDX)
def remove_repetitions(sentence):
    tokens = sentence.split()
    previous_token = None
    filtered_tokens = []
    for token in tokens:
        if token != previous_token:
            filtered_tokens.append(token)
            previous_token = token
    return ' '.join(filtered_tokens)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_adamw.to(device)
def test_and_evaluate(model, test_dataloader, loss_fn, vocab_en, vocab_fr, num_examples=10):
    model.eval()
    total_test_loss = 0
    examples_printed = 0
    meteor = evaluate.load('meteor')
    references = []
    candidates = []
    with torch.no_grad():
      for src, tgt in test_dataloader:
          src, tgt = src.to(device), tgt.to(device)
          src = src.T
          tgt_inp = tgt.T
          tgt_input = tgt_inp[:-1, :]
          tgt_output = tgt_inp[1:, :]
          src_seq_length = src.shape[0]
          tgt_seq_length = tgt_input.shape[0]
          src_mask = torch.zeros(src_seq_length, src_seq_length).to(device).type(torch.bool)
          tgt_mask = model.generate_square_subsequent_mask(tgt_seq_length).to(device)
          src_padding_mask = (src == PAD_IDX).transpose(0, 1).to(device)
          tgt_padding_mask = (tgt_input == PAD_IDX).transpose(0, 1).to(device)

          logits = model(src, tgt_input, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask)
          loss = loss_fn(logits.view(-1, logits.shape[-1]), tgt_output.reshape(-1))
          total_test_loss += loss.item()
          if examples_printed < num_examples:
              predictions = logits.argmax(dim=-1)
              for i in range(predictions.size(1)):
                  if examples_printed >= num_examples:
                      break
                  # input_sentence = ' '.join([vocab_en.get_itos()[idx] for idx in src[:, i] if idx not in (vocab_en['PAD_IDX'], vocab_en['<BOS>'], vocab_en['<EOS>'])])
                  input_sentence = ' '.join([vocab_en.get_itos()[idx] for idx in src[:, i] if idx not in (vocab_en['<pad>'], vocab_en['<bos>'], vocab_en['<eos>'])])
                  target_sentence = ' '.join([vocab_fr.get_itos()[idx] for idx in tgt[:, i] if idx not in (vocab_fr['<pad>'], vocab_fr['<bos>'], vocab_fr['<eos>'])])
                  # predicted_sentence = ' '.join([vocab_fr.get_itos()[idx] for idx in predictions[:, i] if idx not in (vocab_fr['<pad>'], vocab_fr['<bos>'], vocab_fr['<eos>'])])
                  # translated
                  predicted_sentence = remove_repetitions(' '.join([vocab_fr.get_itos()[idx] for idx in predictions[:, i] if idx not in (vocab_fr['<pad>'], vocab_fr['<bos>'], vocab_fr['<eos>'])]))
                  predicted_sentence_with_repititions = ' '.join([vocab_fr.get_itos()[idx] for idx in predictions[:, i] if idx not in (vocab_fr['<pad>'], vocab_fr['<bos>'], vocab_fr['<eos>'])])
                  #print(f'Input: {input_sentence}')
                  #print(f'Target: {target_sentence}')
                  #print(f'Predicted: {predicted_sentence}')
                  #print(f'Predicted with repetitions: {predicted_sentence_with_repititions}')
                 # print()
                  examples_printed += 1
                  references.append([target_sentence])
                  candidates.append(predicted_sentence)
    meteor_score = meteor.compute(predictions=candidates, references=references)
    print(f'Meteor Score: {meteor_score}')
    avg_test_loss = total_test_loss / len(test_dataloader)
    print(f'Test Loss: {avg_test_loss:.4f}')
    return avg_test_loss
test_loss_adamw = test_and_evaluate(model_adamw, test_dataloader, seq2seq_loss_adamw, vocab_en, vocab_fr)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:5109: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


Meteor Score: {'meteor': 0.11332490852304318}
Test Loss: 3.1401


In [ ]:
model_adamw_c.load_state_dict(torch.load('/content/drive/MyDrive/Adamw/absolute_positional_encoding/crossentropywithlabelsmoothning/model_state_adamw_c.pkl'))

<All keys matched successfully>

In [ ]:
from torchtext.data.metrics import bleu_score
import evaluate
seq2seq_loss_adamw_ls = nn.CrossEntropyLoss(ignore_index=PAD_IDX,label_smoothing=0.5)
def remove_repetitions(sentence):
    tokens = sentence.split()
    previous_token = None
    filtered_tokens = []
    for token in tokens:
        if token != previous_token:
            filtered_tokens.append(token)
            previous_token = token
    return ' '.join(filtered_tokens)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_adamw_c.to(device)
def test_and_evaluate(model, test_dataloader, loss_fn, vocab_en, vocab_fr, num_examples=10):
    model.eval()
    total_test_loss = 0
    examples_printed = 0
    meteor = evaluate.load('meteor')
    references = []
    candidates = []
    with torch.no_grad():
      for src, tgt in test_dataloader:
          src, tgt = src.to(device), tgt.to(device)
          src = src.T
          tgt_inp = tgt.T
          tgt_input = tgt_inp[:-1, :]
          tgt_output = tgt_inp[1:, :]
          src_seq_length = src.shape[0]
          tgt_seq_length = tgt_input.shape[0]
          src_mask = torch.zeros(src_seq_length, src_seq_length).to(device).type(torch.bool)
          tgt_mask = model.generate_square_subsequent_mask(tgt_seq_length).to(device)
          src_padding_mask = (src == PAD_IDX).transpose(0, 1).to(device)
          tgt_padding_mask = (tgt_input == PAD_IDX).transpose(0, 1).to(device)

          logits = model(src, tgt_input, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask)
          loss = loss_fn(logits.view(-1, logits.shape[-1]), tgt_output.reshape(-1))
          total_test_loss += loss.item()
          if examples_printed < num_examples:
              predictions = logits.argmax(dim=-1)
              for i in range(predictions.size(1)):
                  if examples_printed >= num_examples:
                      break
                  # input_sentence = ' '.join([vocab_en.get_itos()[idx] for idx in src[:, i] if idx not in (vocab_en['PAD_IDX'], vocab_en['<BOS>'], vocab_en['<EOS>'])])
                  input_sentence = ' '.join([vocab_en.get_itos()[idx] for idx in src[:, i] if idx not in (vocab_en['<pad>'], vocab_en['<bos>'], vocab_en['<eos>'])])
                  target_sentence = ' '.join([vocab_fr.get_itos()[idx] for idx in tgt[:, i] if idx not in (vocab_fr['<pad>'], vocab_fr['<bos>'], vocab_fr['<eos>'])])
                  # predicted_sentence = ' '.join([vocab_fr.get_itos()[idx] for idx in predictions[:, i] if idx not in (vocab_fr['<pad>'], vocab_fr['<bos>'], vocab_fr['<eos>'])])
                  # translated
                  predicted_sentence = remove_repetitions(' '.join([vocab_fr.get_itos()[idx] for idx in predictions[:, i] if idx not in (vocab_fr['<pad>'], vocab_fr['<bos>'], vocab_fr['<eos>'])]))
                  predicted_sentence_with_repititions = ' '.join([vocab_fr.get_itos()[idx] for idx in predictions[:, i] if idx not in (vocab_fr['<pad>'], vocab_fr['<bos>'], vocab_fr['<eos>'])])
                  #print(f'Input: {input_sentence}')
                  #print(f'Target: {target_sentence}')
                  #print(f'Predicted: {predicted_sentence}')
                  #print(f'Predicted with repetitions: {predicted_sentence_with_repititions}')
                  #print()
                  examples_printed += 1
                  references.append([target_sentence])
                  candidates.append(predicted_sentence)
    meteor_score = meteor.compute(predictions=candidates, references=references)
    print(f'Meteor Score: {meteor_score}')
    avg_test_loss = total_test_loss / len(test_dataloader)
    print(f'Test Loss: {avg_test_loss:.4f}')
    return avg_test_loss
test_loss_adamw = test_and_evaluate(model_adamw_c, test_dataloader, seq2seq_loss_adamw_ls, vocab_en, vocab_fr)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:5109: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


Meteor Score: {'meteor': 0.08435759227032555}
Test Loss: 7.6610


In [12]:
model_adamw_n.load_state_dict(torch.load('/content/drive/MyDrive/Adamw/absolute_positional_encoding/sparsecategoricalcrossentropy/model_metrics_adamw_n.pt'))

<All keys matched successfully>

In [14]:
from torchtext.data.metrics import bleu_score
import evaluate
seq2seq_loss_adamw_n = nn.NLLLoss(ignore_index=PAD_IDX)
def remove_repetitions(sentence):
    tokens = sentence.split()
    previous_token = None
    filtered_tokens = []
    for token in tokens:
        if token != previous_token:
            filtered_tokens.append(token)
            previous_token = token
    return ' '.join(filtered_tokens)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_adamw_n.to(device)
def test_and_evaluate(model, test_dataloader, loss_fn, vocab_en, vocab_fr, num_examples=10):
    model.eval()
    total_test_loss = 0
    examples_printed = 0
    meteor = evaluate.load('meteor')
    references = []
    candidates = []
    with torch.no_grad():
      for src, tgt in test_dataloader:
          src, tgt = src.to(device), tgt.to(device)
          src = src.T
          tgt_inp = tgt.T
          tgt_input = tgt_inp[:-1, :]
          tgt_output = tgt_inp[1:, :]
          src_seq_length = src.shape[0]
          tgt_seq_length = tgt_input.shape[0]
          src_mask = torch.zeros(src_seq_length, src_seq_length).to(device).type(torch.bool)
          tgt_mask = model.generate_square_subsequent_mask(tgt_seq_length).to(device)
          src_padding_mask = (src == PAD_IDX).transpose(0, 1).to(device)
          tgt_padding_mask = (tgt_input == PAD_IDX).transpose(0, 1).to(device)

          logits = model(src, tgt_input, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask)
          logits = torch.nn.functional.log_softmax(logits, dim=-1)
          loss = loss_fn(logits.view(-1, logits.shape[-1]), tgt_output.reshape(-1))
          total_test_loss += loss.item()
          if examples_printed < num_examples:
              predictions = logits.argmax(dim=-1)
              for i in range(predictions.size(1)):
                  if examples_printed >= num_examples:
                      break
                  # input_sentence = ' '.join([vocab_en.get_itos()[idx] for idx in src[:, i] if idx not in (vocab_en['PAD_IDX'], vocab_en['<BOS>'], vocab_en['<EOS>'])])
                  input_sentence = ' '.join([vocab_en.get_itos()[idx] for idx in src[:, i] if idx not in (vocab_en['<pad>'], vocab_en['<bos>'], vocab_en['<eos>'])])
                  target_sentence = ' '.join([vocab_fr.get_itos()[idx] for idx in tgt[:, i] if idx not in (vocab_fr['<pad>'], vocab_fr['<bos>'], vocab_fr['<eos>'])])
                  # predicted_sentence = ' '.join([vocab_fr.get_itos()[idx] for idx in predictions[:, i] if idx not in (vocab_fr['<pad>'], vocab_fr['<bos>'], vocab_fr['<eos>'])])
                  # translated
                  predicted_sentence = remove_repetitions(' '.join([vocab_fr.get_itos()[idx] for idx in predictions[:, i] if idx not in (vocab_fr['<pad>'], vocab_fr['<bos>'], vocab_fr['<eos>'])]))
                  predicted_sentence_with_repititions = ' '.join([vocab_fr.get_itos()[idx] for idx in predictions[:, i] if idx not in (vocab_fr['<pad>'], vocab_fr['<bos>'], vocab_fr['<eos>'])])
                  #print(f'Input: {input_sentence}')
                  #print(f'Target: {target_sentence}')
                  #print(f'Predicted: {predicted_sentence}')
                  #print(f'Predicted with repetitions: {predicted_sentence_with_repititions}')
                  #print()
                  examples_printed += 1
                  references.append([target_sentence])
                  candidates.append(predicted_sentence)
    meteor_score = meteor.compute(predictions=candidates, references=references)
    print(f'Meteor Score: {meteor_score}')
    avg_test_loss = total_test_loss / len(test_dataloader)
    print(f'Test Loss: {avg_test_loss:.4f}')
    return avg_test_loss
test_loss_adamw = test_and_evaluate(model_adamw_n, test_dataloader, seq2seq_loss_adamw_n, vocab_en, vocab_fr)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:5109: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


Meteor Score: {'meteor': 0.1059049926012164}
Test Loss: 3.4776


In [ ]:
model_sgd.load_state_dict(torch.load('/content/drive/MyDrive/SGD/absolute_positional_encoding/crossentropyloss/model_sgd_state.pkl'))

<All keys matched successfully>

In [ ]:
from torchtext.data.metrics import bleu_score
import evaluate
seq2seq_loss_sgd = nn.CrossEntropyLoss(ignore_index=PAD_IDX)
def remove_repetitions(sentence):
    tokens = sentence.split()
    previous_token = None
    filtered_tokens = []
    for token in tokens:
        if token != previous_token:
            filtered_tokens.append(token)
            previous_token = token
    return ' '.join(filtered_tokens)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_sgd.to(device)
def test_and_evaluate(model, test_dataloader, loss_fn, vocab_en, vocab_fr, num_examples=10):
    model.eval()
    total_test_loss = 0
    examples_printed = 0
    meteor = evaluate.load('meteor')
    references = []
    candidates = []
    with torch.no_grad():
      for src, tgt in test_dataloader:
          src, tgt = src.to(device), tgt.to(device)
          src = src.T
          tgt_inp = tgt.T
          tgt_input = tgt_inp[:-1, :]
          tgt_output = tgt_inp[1:, :]
          src_seq_length = src.shape[0]
          tgt_seq_length = tgt_input.shape[0]
          src_mask = torch.zeros(src_seq_length, src_seq_length).to(device).type(torch.bool)
          tgt_mask = model.generate_square_subsequent_mask(tgt_seq_length).to(device)
          src_padding_mask = (src == PAD_IDX).transpose(0, 1).to(device)
          tgt_padding_mask = (tgt_input == PAD_IDX).transpose(0, 1).to(device)

          logits = model(src, tgt_input, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask)
          loss = loss_fn(logits.view(-1, logits.shape[-1]), tgt_output.reshape(-1))
          total_test_loss += loss.item()
          if examples_printed < num_examples:
              predictions = logits.argmax(dim=-1)
              for i in range(predictions.size(1)):
                  if examples_printed >= num_examples:
                      break
                  # input_sentence = ' '.join([vocab_en.get_itos()[idx] for idx in src[:, i] if idx not in (vocab_en['PAD_IDX'], vocab_en['<BOS>'], vocab_en['<EOS>'])])
                  input_sentence = ' '.join([vocab_en.get_itos()[idx] for idx in src[:, i] if idx not in (vocab_en['<pad>'], vocab_en['<bos>'], vocab_en['<eos>'])])
                  target_sentence = ' '.join([vocab_fr.get_itos()[idx] for idx in tgt[:, i] if idx not in (vocab_fr['<pad>'], vocab_fr['<bos>'], vocab_fr['<eos>'])])
                  # predicted_sentence = ' '.join([vocab_fr.get_itos()[idx] for idx in predictions[:, i] if idx not in (vocab_fr['<pad>'], vocab_fr['<bos>'], vocab_fr['<eos>'])])
                  # translated
                  predicted_sentence = remove_repetitions(' '.join([vocab_fr.get_itos()[idx] for idx in predictions[:, i] if idx not in (vocab_fr['<pad>'], vocab_fr['<bos>'], vocab_fr['<eos>'])]))
                  predicted_sentence_with_repititions = ' '.join([vocab_fr.get_itos()[idx] for idx in predictions[:, i] if idx not in (vocab_fr['<pad>'], vocab_fr['<bos>'], vocab_fr['<eos>'])])
                  #print(f'Input: {input_sentence}')
                  #print(f'Target: {target_sentence}')
                  #print(f'Predicted: {predicted_sentence}')
                  #print(f'Predicted with repetitions: {predicted_sentence_with_repititions}')
                  #print()
                  examples_printed += 1
                  references.append([target_sentence])
                  candidates.append(predicted_sentence)
    meteor_score = meteor.compute(predictions=candidates, references=references)
    print(f'Meteor Score: {meteor_score}')
    avg_test_loss = total_test_loss / len(test_dataloader)
    print(f'Test Loss: {avg_test_loss:.4f}')
    return avg_test_loss
test_loss_sgd = test_and_evaluate(model_sgd, test_dataloader, seq2seq_loss_sgd, vocab_en, vocab_fr)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:5109: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


Meteor Score: {'meteor': 0.050421514673167724}
Test Loss: 6.8781


In [ ]:
model_sgd_ls.load_state_dict(torch.load('/content/drive/MyDrive/SGD/absolute_positional_encoding/crossentropywithlabelsmoothning/model_sgd_ls_state.pkl'))

<All keys matched successfully>

In [ ]:
from torchtext.data.metrics import bleu_score
import evaluate
seq2seq_loss_sgd_ls = nn.CrossEntropyLoss(ignore_index=PAD_IDX, label_smoothing=0.1)
def remove_repetitions(sentence):
    tokens = sentence.split()
    previous_token = None
    filtered_tokens = []
    for token in tokens:
        if token != previous_token:
            filtered_tokens.append(token)
            previous_token = token
    return ' '.join(filtered_tokens)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_sgd_ls.to(device)
def test_and_evaluate(model, test_dataloader, loss_fn, vocab_en, vocab_fr, num_examples=10):
    model.eval()
    total_test_loss = 0
    examples_printed = 0
    meteor = evaluate.load('meteor')
    references = []
    candidates = []
    with torch.no_grad():
      for src, tgt in test_dataloader:
          src, tgt = src.to(device), tgt.to(device)
          src = src.T
          tgt_inp = tgt.T
          tgt_input = tgt_inp[:-1, :]
          tgt_output = tgt_inp[1:, :]
          src_seq_length = src.shape[0]
          tgt_seq_length = tgt_input.shape[0]
          src_mask = torch.zeros(src_seq_length, src_seq_length).to(device).type(torch.bool)
          tgt_mask = model.generate_square_subsequent_mask(tgt_seq_length).to(device)
          src_padding_mask = (src == PAD_IDX).transpose(0, 1).to(device)
          tgt_padding_mask = (tgt_input == PAD_IDX).transpose(0, 1).to(device)

          logits = model(src, tgt_input, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask)
          loss = loss_fn(logits.view(-1, logits.shape[-1]), tgt_output.reshape(-1))
          total_test_loss += loss.item()
          if examples_printed < num_examples:
              predictions = logits.argmax(dim=-1)
              for i in range(predictions.size(1)):
                  if examples_printed >= num_examples:
                      break
                  # input_sentence = ' '.join([vocab_en.get_itos()[idx] for idx in src[:, i] if idx not in (vocab_en['PAD_IDX'], vocab_en['<BOS>'], vocab_en['<EOS>'])])
                  input_sentence = ' '.join([vocab_en.get_itos()[idx] for idx in src[:, i] if idx not in (vocab_en['<pad>'], vocab_en['<bos>'], vocab_en['<eos>'])])
                  target_sentence = ' '.join([vocab_fr.get_itos()[idx] for idx in tgt[:, i] if idx not in (vocab_fr['<pad>'], vocab_fr['<bos>'], vocab_fr['<eos>'])])
                  # predicted_sentence = ' '.join([vocab_fr.get_itos()[idx] for idx in predictions[:, i] if idx not in (vocab_fr['<pad>'], vocab_fr['<bos>'], vocab_fr['<eos>'])])
                  # translated
                  predicted_sentence = remove_repetitions(' '.join([vocab_fr.get_itos()[idx] for idx in predictions[:, i] if idx not in (vocab_fr['<pad>'], vocab_fr['<bos>'], vocab_fr['<eos>'])]))
                  predicted_sentence_with_repititions = ' '.join([vocab_fr.get_itos()[idx] for idx in predictions[:, i] if idx not in (vocab_fr['<pad>'], vocab_fr['<bos>'], vocab_fr['<eos>'])])
                  #print(f'Input: {input_sentence}')
                  #print(f'Target: {target_sentence}')
                  #print(f'Predicted: {predicted_sentence}')
                  #print(f'Predicted with repetitions: {predicted_sentence_with_repititions}')
                  #print()
                  examples_printed += 1
                  references.append([target_sentence])
                  candidates.append(predicted_sentence)
    meteor_score = meteor.compute(predictions=candidates, references=references)
    print(f'Meteor Score: {meteor_score}')
    avg_test_loss = total_test_loss / len(test_dataloader)
    print(f'Test Loss: {avg_test_loss:.4f}')
    return avg_test_loss
test_loss_sgd_ls = test_and_evaluate(model_sgd_ls, test_dataloader, seq2seq_loss_sgd_ls, vocab_en, vocab_fr)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Meteor Score: {'meteor': 0.04752838603414884}
Test Loss: 7.3659


In [16]:
model_sgd_n.load_state_dict(torch.load('/content/drive/MyDrive/SGD/absolute_positional_encoding/sparsecategoricalcrossentropy/model_sgd_n_state.pkl'))

<All keys matched successfully>

In [17]:
from torchtext.data.metrics import bleu_score
import evaluate
seq2seq_loss_sgd_n = nn.NLLLoss(ignore_index=PAD_IDX)
def remove_repetitions(sentence):
    tokens = sentence.split()
    previous_token = None
    filtered_tokens = []
    for token in tokens:
        if token != previous_token:
            filtered_tokens.append(token)
            previous_token = token
    return ' '.join(filtered_tokens)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_sgd_n.to(device)
def test_and_evaluate(model, test_dataloader, loss_fn, vocab_en, vocab_fr, num_examples=10):
    model.eval()
    total_test_loss = 0
    examples_printed = 0
    meteor = evaluate.load('meteor')
    references = []
    candidates = []
    with torch.no_grad():
      for src, tgt in test_dataloader:
          src, tgt = src.to(device), tgt.to(device)
          src = src.T
          tgt_inp = tgt.T
          tgt_input = tgt_inp[:-1, :]
          tgt_output = tgt_inp[1:, :]
          src_seq_length = src.shape[0]
          tgt_seq_length = tgt_input.shape[0]
          src_mask = torch.zeros(src_seq_length, src_seq_length).to(device).type(torch.bool)
          tgt_mask = model.generate_square_subsequent_mask(tgt_seq_length).to(device)
          src_padding_mask = (src == PAD_IDX).transpose(0, 1).to(device)
          tgt_padding_mask = (tgt_input == PAD_IDX).transpose(0, 1).to(device)

          logits = model(src, tgt_input, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask)
          logits = torch.nn.functional.log_softmax(logits, dim=-1)
          loss = loss_fn(logits.view(-1, logits.shape[-1]), tgt_output.reshape(-1))
          total_test_loss += loss.item()
          if examples_printed < num_examples:
              predictions = logits.argmax(dim=-1)
              for i in range(predictions.size(1)):
                  if examples_printed >= num_examples:
                      break
                  # input_sentence = ' '.join([vocab_en.get_itos()[idx] for idx in src[:, i] if idx not in (vocab_en['PAD_IDX'], vocab_en['<BOS>'], vocab_en['<EOS>'])])
                  input_sentence = ' '.join([vocab_en.get_itos()[idx] for idx in src[:, i] if idx not in (vocab_en['<pad>'], vocab_en['<bos>'], vocab_en['<eos>'])])
                  target_sentence = ' '.join([vocab_fr.get_itos()[idx] for idx in tgt[:, i] if idx not in (vocab_fr['<pad>'], vocab_fr['<bos>'], vocab_fr['<eos>'])])
                  # predicted_sentence = ' '.join([vocab_fr.get_itos()[idx] for idx in predictions[:, i] if idx not in (vocab_fr['<pad>'], vocab_fr['<bos>'], vocab_fr['<eos>'])])
                  # translated
                  predicted_sentence = remove_repetitions(' '.join([vocab_fr.get_itos()[idx] for idx in predictions[:, i] if idx not in (vocab_fr['<pad>'], vocab_fr['<bos>'], vocab_fr['<eos>'])]))
                  predicted_sentence_with_repititions = ' '.join([vocab_fr.get_itos()[idx] for idx in predictions[:, i] if idx not in (vocab_fr['<pad>'], vocab_fr['<bos>'], vocab_fr['<eos>'])])
                  #print(f'Input: {input_sentence}')
                  #print(f'Target: {target_sentence}')
                  #print(f'Predicted: {predicted_sentence}')
                  #print(f'Predicted with repetitions: {predicted_sentence_with_repititions}')
                  #print()
                  examples_printed += 1
                  references.append([target_sentence])
                  candidates.append(predicted_sentence)
    meteor_score = meteor.compute(predictions=candidates, references=references)
    print(f'Meteor Score: {meteor_score}')
    avg_test_loss = total_test_loss / len(test_dataloader)
    print(f'Test Loss: {avg_test_loss:.4f}')
    return avg_test_loss
test_loss_sgd_ls = test_and_evaluate(model_sgd_n, test_dataloader, seq2seq_loss_sgd_n, vocab_en, vocab_fr)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:5109: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


Meteor Score: {'meteor': 0.06805303595540604}
Test Loss: 6.9419
